# How to upload and publish a research product on Zenodo through the Zenodo REST API

The **Zenodo REST API** currently supports:

 - **Deposit** — upload and publishing of research outputs (identical to functionality available in the user interface).
 - **Records** — search published records.
 - **Files** — download/upload of files
 
The base URL of the API is https://zenodo.org/api/ (or https://sandbox.zenodo.org/api/ for testing purposes).

## *Depositions* entity
The Deposition resource is used for uploading and editing records on Zenodo.

 - List all the depositions for the currently authenticated user
    
    `GET /api/deposit/depositions`
    
 - Create a new deposition resource
    
    `POST /api/deposit/depositions`
    
 - Retrieve a single deposition resource
    
    `GET /api/deposit/depositions/:id`
  
 - Update an existing deposition resource
    
    `PUT /api/deposit/depositions/:id`
     
 - Delete an existing deposition resource. Note, only unpublished depositions may be deleted.
    
    `DELETE /api/deposit/depositions/:id`

## Deposition *files*
The Deposition file resource is used for uploading and editing files of a deposition on Zenodo.

 - List all the deposition files for a given deposition
    
    `GET /api/deposit/depositions/:id/files`
    
 - Upload a new file
   
   `POST /api/deposit/depositions/:id/files`
    
 - Sort the files for a deposition. By default, the first file is shown in the file preview
   
   `PUT /api/deposit/depositions/:id/files`
  
 - Retrieve a single deposition file
   
   `GET /api/deposit/depositions/:id/files/:file_id`
     
 - Update a deposition file resource. Currently the only use is renaming an already uploaded file. If you one to replace the actual file, please delete the file and upload a new file.
   
   `PUT /api/deposit/depositions/:id/files/:file_id`
 
 - Delete an existing deposition file resource. Note, only deposition files for unpublished depositions may be deleted.
   
   `DELETE /api/deposit/depositions/:id/files/:file_id`

## Deposition *actions*

- Publish a deposition. Note, once a deposition is published, you can no longer delete it.
    
    `POST /api/deposit/depositions/:id/actions/publish`
    
 - Unlock already submitted deposition for editing
   
   `POST /api/deposit/depositions/:id/actions/edit`
    
 - Discard changes in the current editing session
   
   `POST /api/deposit/depositions/:id/actions/discard`
  
 - Create a new version of a deposition
   
   `POST /api/deposit/depositions/:id/actions/newversion`

## *Records* entity
The Records resource is used to search through published records.

- List all open access records: [here](https://developers.zenodo.org/#list36) more info about the query parameters
    
    `GET /api/records/`
    
 - Retrieve a single record
   
   `GET /api/records/:id`
    
 - Discard changes in the current editing session
   
   `POST /api/deposit/depositions/:id/actions/discard`
  
 - Create a new version of a deposition
   
   `POST /api/deposit/depositions/:id/actions/newversion`



As first step, let's import the `requests` module to handle HTTP requests.

In [9]:
import requests

If you wish to test the API without publishing any product online, you can use the **sandbox environment** where you can test your API integration during development. The sandbox environment is available at http://sandbox.zenodo.org.

Please note that the sandbox environment can be cleaned at anytime. Also, the sandbox environment will issue test DOIs using the `10.5072` prefix instead of the Zenodo’s normal prefix `10.5281`.

In [10]:
base_url = "https://sandbox.zenodo.org/api/"
#base_url = "https://zenodo.org/api/"

All the  APIs  require an **access token**. Create a token and replace ACCESS_TOKEN with your newly created personal access token.

**Note:** you need two different tokens for working in the sandbox and official environments.

To create a personal access token:

  - **Register** for a **Zenodo account** if you don’t already have one.
  - Go to your Applications to **create a new token**.
  - Select the **OAuth scopes** you need (you need *deposit:write* and *deposit:actions*).
  
**Note**
  - **deposit:write** ---> Grants write access to depositions, but does not allow publishing the upload.
  - **deposit:actions** ---> Grants access to publish, edit and discard edits for depositions.


In [11]:
#ACCESS_TOKEN = 'EpjJtbio84ooKJ7jdnKL3RoQ7uoO6dMzTSw3VCMtOo6lS7Fu89ypH6QP2OE0' #official
ACCESS_TOKEN = 'J7kaMPAhcu1EG1JX1PlhiTGuXLxBGQ0NyeqZnX9465jARs6d8YvjMEPlC9hq' #sandbox

### STEP 1: Let’s create a new empty upload

In [12]:
headers = {"Content-Type": "application/json"}
params = {'access_token': ACCESS_TOKEN}
r = requests.post(base_url+'/deposit/depositions',
                   params=params,
                   json={},
                   headers=headers)
r.json()

{'conceptrecid': '1176679',
 'created': '2023-03-27T13:39:32.328035+00:00',
 'files': [],
 'id': 1176680,
 'links': {'bucket': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250',
  'discard': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/actions/discard',
  'edit': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/actions/edit',
  'files': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/files',
  'html': 'https://sandbox.zenodo.org/deposit/1176680',
  'latest_draft': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680',
  'latest_draft_html': 'https://sandbox.zenodo.org/deposit/1176680',
  'publish': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/actions/publish',
  'self': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680'},
 'metadata': {'prereserve_doi': {'doi': '10.5072/zenodo.1176680',
   'recid': 1176680}},
 'modified': '2023-03-27T13:39:32.328045+00:00',
 'owner': 142736,
 'record_id': 11

The deposition **id** represents the deposition identifier to be used in the next steps, so let's extract it from the previous json response.

In [13]:
deposition_id = r.json()['id']
deposition_id

1176680

Similarly, to use the new files API we will need to do a PUT request to the bucket link. The bucket is a folder-like object storing the files of our record and can be found under the 'links' key in our records metadata.

In [14]:
bucket_url = r.json()["links"]["bucket"]
bucket_url

'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250'

### STEP 2: Let’s upload a new file

To upload a new file with the **old API**: `POST /api/deposit/depositions/:id/files`
 
Example:

```
filename = "TEST.txt"
path = "/home/jovyan/work/%s" % filename
data = {'name': filename}
deposition_id = r.json()['id']
files = {'file': open(path, 'rb')}
r = requests.post(base_url+'/deposit/depositions/%s/files' % deposition_id,
                   params={'access_token': ACCESS_TOKEN}, data=data,
                   files=files)
```

In this case, we are going to use the **new API**, which is significantly more perfomant and supports much larger file sizes. While the older API supports 100MB per file, the new one has no size limitation.

The target URL is a combination of the bucket link with the desired filename seperated by a slash: `PUT /api/files/:id/filename`


In [15]:
''' New API '''
filename = "TEST.txt"
path = "/home/jovyan/work/%s" % filename

with open(path, "rb") as fp:
    r = requests.put(
        "%s/%s" % (bucket_url, filename),
        data=fp,
        params=params,
    )
r.json()

{'mimetype': 'text/plain',
 'updated': '2023-03-27T13:42:11.801738+00:00',
 'links': {'self': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250/TEST.txt',
  'version': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250/TEST.txt?versionId=8b962084-d2ec-4438-88ea-e32e8e5c219d',
  'uploads': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250/TEST.txt?uploads'},
 'is_head': True,
 'created': '2023-03-27T13:42:11.795535+00:00',
 'checksum': 'md5:c0c93e7112ac277748532616405bb875',
 'version_id': '8b962084-d2ec-4438-88ea-e32e8e5c219d',
 'delete_marker': False,
 'key': 'TEST.txt',
 'size': 9}

### STEP 3: Let's add some metadata

To do so, we need to update the **metadata** field of the desosition resource (see output of step 1).

For a comprehensive description of the available metadata attributes, please refer to the **Deposit metadata** table at the following link:
https://developers.zenodo.org/#representation

In [16]:
import json
data = {
     'metadata': {
         'title': 'My first upload',
         'upload_type': 'other',
         'description': 'This is my first upload',
         'creators': [{'name': 'Fabrizio, Antonio','affiliation': 'CMCC'}],
         'access_right': 'open',
         'license':'cc-by',
         'keywords': ["Key_A", "Key_B"]
     }
 }
r = requests.put(base_url+'/deposit/depositions/%s' % deposition_id,
                  params={'access_token': ACCESS_TOKEN}, data=json.dumps(data),
                  headers=headers)
r.json()

{'conceptrecid': '1176679',
 'created': '2023-03-27T13:39:32.328035+00:00',
 'doi': '',
 'doi_url': 'https://doi.org/',
 'files': [{'checksum': 'c0c93e7112ac277748532616405bb875',
   'filename': 'TEST.txt',
   'filesize': 9,
   'id': '534ac5de-4e5b-40de-bb15-e5affa57e814',
   'links': {'download': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250/TEST.txt',
    'self': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/files/534ac5de-4e5b-40de-bb15-e5affa57e814'}}],
 'id': 1176680,
 'links': {'bucket': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250',
  'discard': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/actions/discard',
  'edit': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/actions/edit',
  'files': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/files',
  'html': 'https://sandbox.zenodo.org/deposit/1176680',
  'latest_draft': 'https://sandbox.zenodo.org/api/deposit/depositions/

### STEP 4: We’re ready to publish

In [17]:
r = requests.post(base_url+'/deposit/depositions/%s/actions/publish' % deposition_id,
                      params={'access_token': ACCESS_TOKEN} )

This is the corresponding json response

In [18]:
r.json()

{'conceptdoi': '10.5072/zenodo.1176679',
 'conceptrecid': '1176679',
 'created': '2023-03-27T13:39:32.328035+00:00',
 'doi': '10.5072/zenodo.1176680',
 'doi_url': 'https://doi.org/10.5072/zenodo.1176680',
 'files': [{'checksum': 'c0c93e7112ac277748532616405bb875',
   'filename': 'TEST.txt',
   'filesize': 9,
   'id': '534ac5de-4e5b-40de-bb15-e5affa57e814',
   'links': {'download': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250/TEST.txt',
    'self': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/files/534ac5de-4e5b-40de-bb15-e5affa57e814'}}],
 'id': 1176680,
 'links': {'badge': 'https://sandbox.zenodo.org/badge/doi/10.5072/zenodo.1176680.svg',
  'bucket': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250',
  'conceptbadge': 'https://sandbox.zenodo.org/badge/doi/10.5072/zenodo.1176679.svg',
  'conceptdoi': 'https://doi.org/10.5072/zenodo.1176679',
  'doi': 'https://doi.org/10.5072/zenodo.1176680',
  'latest': 'https://san

And this is the DOI assigned to our record

In [19]:
record_doi = r.json()["doi"]
record_doi

'10.5072/zenodo.1176680'

In [20]:
record_doi_url = r.json()["doi_url"]
record_doi_url

'https://doi.org/10.5072/zenodo.1176680'

We can also get the list of our records. The response is the list of all depositions for the currently authenticated user, as described [here](https://developers.zenodo.org/#list).

In [21]:
depositions_list = requests.get(base_url+'/deposit/depositions',
                  params={'access_token': ACCESS_TOKEN})
depositions_list.json()

[{'conceptdoi': '10.5072/zenodo.1176679',
  'conceptrecid': '1176679',
  'created': '2023-03-27T13:39:32.328035',
  'doi': '10.5072/zenodo.1176680',
  'doi_url': 'https://doi.org/10.5072/zenodo.1176680',
  'files': [{'checksum': 'c0c93e7112ac277748532616405bb875',
    'filename': 'TEST.txt',
    'filesize': 9,
    'id': '534ac5de-4e5b-40de-bb15-e5affa57e814',
    'links': {'download': 'https://sandbox.zenodo.org/api/files/b20576a3-9b5e-4f08-9a2f-19a5f68b3a67/TEST.txt',
     'self': 'https://sandbox.zenodo.org/api/deposit/depositions/1176680/files/534ac5de-4e5b-40de-bb15-e5affa57e814'}}],
  'id': 1176680,
  'links': {'badge': 'https://sandbox.zenodo.org/badge/doi/10.5072/zenodo.1176680.svg',
   'bucket': 'https://sandbox.zenodo.org/api/files/b5818227-c6f7-4ed9-9575-5b749cf10250',
   'conceptbadge': 'https://sandbox.zenodo.org/badge/doi/10.5072/zenodo.1176679.svg',
   'conceptdoi': 'https://doi.org/10.5072/zenodo.1176679',
   'discard': 'https://sandbox.zenodo.org/api/deposit/depositions

We can loop through the list and access the metadata (e.g., doi_url)

In [22]:
for d in depositions_list.json():
    print(d["doi_url"])

https://doi.org/10.5072/zenodo.1176680
https://doi.org/10.5072/zenodo.1176587


## Search for some open access records

As an example, we are interested in records related to the **EGI-ACE community**.

As first step, let's check if the community does exist and get its id.

In [54]:
ACCESS_TOKEN = 'EpjJtbio84ooKJ7jdnKL3RoQ7uoO6dMzTSw3VCMtOo6lS7Fu89ypH6QP2OE0' 
params = {'access_token': ACCESS_TOKEN, 'q':'egi-ace'}

r = requests.get('https://zenodo.org/api/communities/',
                  params=params)

r.json()

{'hits': {'hits': [{'updated': '2021-12-01T09:05:51.584735+00:00',
    'id_user': 118732,
    'description': '<p>EGI-ACE is a 30-month project (Jan 2021 - June 2023) with a mission to empower researchers from all disciplines to collaborate in data- and compute-intensive research through free-at-point-of-use services. EGI-ACE delivers the EOSC Compute Platform, a system of federated compute and storage facilities to support processing and analytics for distributed data and compute use cases. The EOSC Compute Platform enables the efficient use of the European Commission (EC) and national funds by integrating cross-border access with national access, maximising the return of investments for stakeholders. The project also delivers Data Spaces, systems of data and applications from different scientific disciplines that are hosted on the EOSC Compute Platform for easy and scalable analysis and exploitation of FAIR datasets.</p>\r\n\r\n<p>The purpose of this&nbsp;community is to share publica

In [53]:
egi_ace_id = r.json()['hits']['hits'][0]['id']
egi_ace_id

'egi-ace'

Now, let's retrieve all the records related to the EGI-ACE community. To do so, we use the `communities` query argument.

In [84]:
ACCESS_TOKEN = 'EpjJtbio84ooKJ7jdnKL3RoQ7uoO6dMzTSw3VCMtOo6lS7Fu89ypH6QP2OE0' 

params = {'access_token': ACCESS_TOKEN, 'communities':egi_ace_id}

r = requests.get('https://zenodo.org/api/records/',
                  params=params)

r.json()

{'aggregations': {'access_right': {'buckets': [{'doc_count': 23,
     'key': 'open'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0},
  'file_type': {'buckets': [{'doc_count': 22, 'key': 'pdf'},
    {'doc_count': 1, 'key': 'pptx'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0},
  'keywords': {'buckets': [{'doc_count': 3, 'key': 'Compute'},
    {'doc_count': 3, 'key': 'EOSC'},
    {'doc_count': 3, 'key': 'Virtual access'},
    {'doc_count': 2, 'key': 'Data'},
    {'doc_count': 2, 'key': 'EGI-ACE'},
    {'doc_count': 2, 'key': 'HPC'},
    {'doc_count': 2, 'key': 'SMS'},
    {'doc_count': 2, 'key': 'engagement'},
    {'doc_count': 1, 'key': 'AAI'},
    {'doc_count': 1, 'key': 'Catalogues'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 31},
  'type': {'buckets': [{'doc_count': 21,
     'key': 'publication',
     'subtype': {'buckets': [{'doc_count': 19, 'key': 'deliverable'},
       {'doc_count': 1, 'key': 'milestone'},
       {'d

Let's extract DOI and title of the returned records.

In [85]:
for record in r.json()['hits']['hits']:
    print('%s - %s' % (record['metadata']['doi'], record['metadata']['title']))

10.5281/zenodo.7670205 - D7.4 Green Computing progress and improvements within EGI-ACE
10.5281/zenodo.7463329 - EGI-ACE D2.8 Technical, Policy and Service Management Integration Report
10.5281/zenodo.7189370 - EGI-ACE D2.7 Technical specifications for compute common services
10.5281/zenodo.6968509 - EGI-ACE D7.3 Final version HPC integration Handbook
10.5281/zenodo.6968566 - EGI-ACE D2.6 Communication and engagement plan
10.5281/zenodo.6944570 - EGI-ACE D2.2 EGI-ACE Strategic Plan
10.5281/zenodo.6783908 - EGI-ACE D1.7 Dissemination and Exploitation Plan
10.5281/zenodo.6602285 - EGI-ACE D1.4 Dissemination and Exploitation Plan
10.5281/zenodo.6602279 - EGI-ACE D2.1 Communication and engagement plan
10.5281/zenodo.6602270 - EGI-ACE D2.3 Technical specifications for compute common services


We can also specify a search query (`q`) using the Elasticsearch query string syntax. For example, we are interested in record about **dissemination**.

In [86]:
ACCESS_TOKEN = 'EpjJtbio84ooKJ7jdnKL3RoQ7uoO6dMzTSw3VCMtOo6lS7Fu89ypH6QP2OE0' 

params = {'access_token': ACCESS_TOKEN, 'communities':egi_ace_id, 'q':'dissemination'}

r = requests.get('https://zenodo.org/api/records/',
                  params=params)

r.json()

{'aggregations': {'access_right': {'buckets': [{'doc_count': 2,
     'key': 'open'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0},
  'file_type': {'buckets': [{'doc_count': 2, 'key': 'pdf'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0},
  'keywords': {'buckets': [{'doc_count': 1, 'key': 'Dissemination'},
    {'doc_count': 1, 'key': 'Exploitation'},
    {'doc_count': 1, 'key': 'dissemination'},
    {'doc_count': 1, 'key': 'exploitation'},
    {'doc_count': 1, 'key': 'key exploitable results'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0},
  'type': {'buckets': [{'doc_count': 2,
     'key': 'publication',
     'subtype': {'buckets': [{'doc_count': 2, 'key': 'deliverable'}],
      'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0}}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0}},
 'hits': {'hits': [{'conceptdoi': '10.5281/zenodo.5126645',
    'conceptrecid': '5126645',
    'created': '202

In [87]:
for record in r.json()['hits']['hits']:
    print('%s - %s' % (record['metadata']['doi'], record['metadata']['title']))

10.5281/zenodo.6602285 - EGI-ACE D1.4 Dissemination and Exploitation Plan
10.5281/zenodo.6783908 - EGI-ACE D1.7 Dissemination and Exploitation Plan
